In [49]:
!pip install ipykernel
!python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"

Installed kernelspec myenv in C:\Users\User\AppData\Roaming\jupyter\kernels\myenv


In [50]:
!pip install Pillow

In [51]:
from PIL import Image
print("Pillow is installed and imported successfully!")

Pillow is installed and imported successfully!


# Initialisation

In [52]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os

In [53]:
# Define directory paths
train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/val'
test_data_dir = 'dataset/test'

# Define image dimensions and batch size
img_height, img_width = 256, 256
batch_size = 32

In [54]:
# Create data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Usually for test data, we don't shuffle
)

# Print the class indices
print("Class indices:", train_generator.class_indices)

Found 2428 images belonging to 3 classes.
Found 671 images belonging to 3 classes.
Found 886 images belonging to 3 classes.
Class indices: {'duck': 0, 'eagle': 1, 'hornbill': 2}


In [55]:
# Load the ResNet50 model with pre-trained ImageNet weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Add a fully connected layer
predictions = Dense(5, activation='softmax')(x)  # Output layer with 5 classes

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with the correct learning rate argument
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [56]:
epochs = 50

model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [ ]:
# Evaluate the model on the validation data
model.evaluate(validation_generator)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.